In [ ]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings("ignore", message=".*Could not infer format.*")
from collections import defaultdict
from difflib import SequenceMatcher
from collections import defaultdict
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import re
import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
#### Input the Data Folder (Where all the files are) and the INFO SHEET (that has all the PK/HIST,etc marked)
DATA_FOLDER = r"C:\Users\shobh\OneDrive\Desktop\Analytics Project\Dataset\test_data"
excel_path = r"C:\Users\shobh\OneDrive\Desktop\Analytics Project\Dataset\information_test_data.xlsx"
xl = pd.ExcelFile(excel_path)
ID_KEYWORDS = ['id', 'key', 'ref', 'dt']

def ends_with_id_keyword(col_name):
    col_name = col_name.lower()
    return any(col_name.endswith(k) for k in ID_KEYWORDS)

In [3]:
def is_datetime_like(series):
    try:
        parsed = pd.to_datetime(series.dropna(), errors='coerce')
        non_null_ratio = parsed.notnull().sum() / len(series.dropna())
        return non_null_ratio >= 0.9
    except Exception:
        return False

In [ ]:
def extract_metadata_features(folder_path):
    records = []

    for file in Path(folder_path).glob("*.csv"):
        table_name = re.sub(r"\.0$", "", file.stem.strip().lower())
        df = pd.read_csv(file, dtype=str, low_memory=False)

        df = df.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == '' else str(x).strip())
        df = df.apply(lambda col: col.apply(lambda x: x.lower() if isinstance(x, str) else x))


        for i, col in enumerate(df.columns):
            col_data = df[col]

            total = len(col_data)
            non_null_count = col_data.notnull().sum()
            null_percent = 100 * (total - non_null_count) / total if total > 0 else 0
            contains_nulls = null_percent > 0

            col_data_nonnull = col_data.dropna()
            unique_ratio = col_data_nonnull.nunique() / non_null_count if non_null_count > 0 else 0
            inferred_dtype = "string"
            avg_val_len = max_val_len = min_val_len = None

            try:
                as_num = pd.to_numeric(col_data_nonnull, errors="coerce")
                if as_num.notnull().all() and not as_num.empty:
                    if (as_num % 1 == 0).all():
                        inferred_dtype = "integer"
                        lengths = as_num.astype(str).str.len()
                    else:
                        inferred_dtype = "float"
                        lengths = as_num.astype(str).str.replace(r"\..*", "", regex=True).str.len()
                else:
                    as_date = pd.to_datetime(col_data_nonnull, errors="coerce")
                    if as_date.notnull().all() and not as_date.empty:
                        inferred_dtype = "datetime"
                        lengths = None
                    else:
                        inferred_dtype = "string"
                        lengths = col_data_nonnull.astype(str).str.len()
            except:
                inferred_dtype = "string"
                lengths = col_data_nonnull.astype(str).str.len()

            if lengths is not None and not lengths.empty:
                avg_val_len = lengths.mean()
                max_val_len = lengths.max()
                min_val_len = lengths.min()
                mode_val_len = lengths.mode().iloc[0] if not lengths.mode().empty else None
            else:
                mode_val_len = None

            record = {
                "table_name": table_name,
                "column_name": col,
                "column_position": i,
                "name_length": len(col),
                "contains_keyword_id": ends_with_id_keyword(col),
                "contains_nulls": contains_nulls,
                "null_percent": round(null_percent, 2),
                "is_unique": unique_ratio == 1.0 if non_null_count > 0 else False,
                "inferred_dtype": inferred_dtype,
                "avg_val_len": avg_val_len,
                "min_val_len": min_val_len,
                "max_val_len": max_val_len,
                "mode_val_len": mode_val_len
            }

            records.append(record)

    return pd.DataFrame(records)

In [5]:
metadata_df = extract_metadata_features(DATA_FOLDER)

C:\Users\shobh\AppData\Local\Temp\ipykernel_16104\1452100316.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == '' else str(x).strip())
C:\Users\shobh\AppData\Local\Temp\ipykernel_16104\1452100316.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == '' else str(x).strip())
C:\Users\shobh\AppData\Local\Temp\ipykernel_16104\1452100316.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == '' else str(x).strip())
C:\Users\shobh\AppData\Local\Temp\ipykernel_16104\1452100316.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if pd.isna(x) or str(x).strip() == '' else str(x).strip())
C:\Users\shobh\AppData\L

In [7]:
pk_indicators = {"PK", "SKPK", "PK/HIST"}

pk_labels = []
for sheet_name in xl.sheet_names:
    df = xl.parse(sheet_name)
    if "Column Name" in df.columns and "Type" in df.columns:
        for _, row in df.iterrows():
            col_name = str(row["Column Name"]).strip().lower()
            label = str(row["Type"]).strip().upper() if pd.notnull(row["Type"]) else ""
            if label in pk_indicators:
                pk_labels.append((sheet_name.lower(), col_name))

In [8]:
metadata_df["table_name_clean"] = metadata_df["table_name"].str.strip().str.lower().str.replace(r"\.0$", "", regex=True)
metadata_df["column_name_clean"] = metadata_df["column_name"].str.strip().str.lower()

In [9]:
pk_set = set(pk_labels)

In [10]:
metadata_df["is_primary_key"] = metadata_df.apply(
    lambda row: 1 if (row["table_name_clean"], row["column_name_clean"]) in pk_set else 0,
    axis=1
)

In [11]:
metadata_df.drop(columns=["table_name_clean", "column_name_clean"], inplace=True)

In [12]:
pk_lookup = defaultdict(set)
for tbl, col in pk_labels:
    pk_lookup[tbl].add(col)

In [13]:
metadata_df["table_name_lower"] = metadata_df["table_name"].str.strip().str.lower().str.replace(r"\.0$", "", regex=True)
metadata_df["column_name_lower"] = metadata_df["column_name"].str.strip().str.lower()

In [14]:
pk_columns_df = metadata_df[metadata_df["is_primary_key"] == 1]
fk_candidate_df = metadata_df.copy()  # include all columns as FK candidates

In [16]:
pk_groups = defaultdict(list)
for _, row in pk_columns_df.iterrows():
    pk_groups[row["table_name_lower"]].append(row["column_name_lower"])

In [ ]:
def profile_columns_by_type(folder_path, metadata_df):
    stats_records = []

    for file in tqdm(Path(folder_path).glob("*.csv"), desc="Processing CSV files"):
        try:
            df = pd.read_csv(file, dtype=str, low_memory=False)
        except Exception as e:
            print(f"Could not read {file.name}: {e}")
            continue

        table_name = re.sub(r"\.0$", "", file.stem.strip().lower())

        # Use cleaned name to match
        subset = metadata_df[metadata_df["table_name_lower"] == table_name]

        for _, row in tqdm(subset.iterrows(), total=len(subset), desc=f"Profiling {table_name}", leave=False):
            col_name = row["column_name"]
            dtype = row["inferred_dtype"]
            stat = {
                "table_name": table_name,
                "column_name": col_name,
                "inferred_dtype": dtype
            }

            if col_name not in df.columns:
                stat["error"] = "missing_column"
                stats_records.append(stat)
                continue

            col_data = df[col_name].dropna()

            try:
                if dtype in ["integer", "float", "floating"]:
                    num_data = pd.to_numeric(col_data, errors='coerce').dropna()
                    if not num_data.empty:
                        stat.update({
                            "min": num_data.min(),
                            "max": num_data.max(),
                            "mean": num_data.mean(),
                            "median": num_data.median(),
                            "std": num_data.std(),
                            "num_unique": num_data.nunique(),
                            "num_nonnull": num_data.count()
                        })

                elif dtype == "string":
                    str_lengths = col_data.astype(str).str.len()
                    stat.update({
                        "min_len": str_lengths.min(),
                        "max_len": str_lengths.max(),
                        "avg_len": str_lengths.mean(),
                        "num_unique": col_data.nunique(),
                        "most_freq_value": col_data.mode().iloc[0] if not col_data.mode().empty else np.nan
                    })

                elif dtype == "datetime":
                    dt_parsed = pd.to_datetime(col_data, errors='coerce').dropna()
                    if not dt_parsed.empty:
                        stat.update({
                            "min_date": dt_parsed.min(),
                            "max_date": dt_parsed.max(),
                            "num_unique_dates": dt_parsed.nunique()
                        })
            except Exception as e:
                stat["error"] = str(e)

            stats_records.append(stat)

    return pd.DataFrame(stats_records)

In [ ]:
def compute_fk_pk_score(pk_row, fk_row, length_tolerance=0.2):
    score = 0
    reasons = []

    if fk_row["null_percent"] >= 100:
        return 0, ["fk_is_100%_null"]
    if pk_row["inferred_dtype"] != fk_row["inferred_dtype"]:
        return 0, ["dtype_mismatch"]

    name_sim = SequenceMatcher(None, pk_row["column_name_lower"], fk_row["column_name_lower"]).ratio()
    if name_sim > 0.7:
        score += 3
        reasons.append(f"name_match:{round(name_sim,2)}")

    score += 2
    reasons.append("dtype_match")

    pk_mode = pk_row.get("mode_val_len")
    fk_mode = fk_row.get("mode_val_len")

    if pk_mode and fk_mode:
        ratio = fk_mode / pk_mode if pk_mode != 0 else 0
        if 0.8 <= ratio <= 1.2:
            score += 2
            reasons.append("val_mode_len_match")


    if pk_row["avg_val_len"] and fk_row["avg_val_len"]:
        ratio = fk_row["avg_val_len"] / pk_row["avg_val_len"]
        if 1 - length_tolerance <= ratio <= 1 + length_tolerance:
            score += 0.5
            reasons.append("avg_len_similar")

    if pk_row["column_position"] < 5:
        score += 0.5
        reasons.append("pk_early")
    if fk_row["column_position"] < 5:
        score += 0.5
        reasons.append("fk_early")

    if fk_row["null_percent"] < 50:
        score += 0.5
        reasons.append("fk_has_few_nulls")

    if pk_row["is_unique"]:
        score += 0.5
        reasons.append("pk_is_unique")
    if not fk_row["is_unique"]:
        score += 0.5
        reasons.append("fk_not_unique")

    return score, reasons

In [ ]:
matches = []

seen_pairs = set()

for _, pk in pk_columns_df.iterrows():
    for _, fk in fk_candidate_df.iterrows():
        if pk["table_name_lower"] == fk["table_name_lower"]:
            continue

        pk_id = (pk["table_name_lower"], pk["column_name_lower"])
        fk_id = (fk["table_name_lower"], fk["column_name_lower"])
        pair_key = tuple(sorted([pk_id, fk_id]))

        if pair_key in seen_pairs:
            continue

        score, reasons = compute_fk_pk_score(pk, fk)
        if score >= 3:
            matches.append({
                "pk_table": pk["table_name"],
                "pk_column": pk["column_name"],
                "fk_table": fk["table_name"],
                "fk_column": fk["column_name"],
                "score": round(score, 2),
                "reasons": ", ".join(reasons)
            })
            seen_pairs.add(pair_key)


In [ ]:
matches_df = pd.DataFrame(matches).sort_values(by="score", ascending=False)

In [ ]:
def canonicalize(val):
    if pd.isnull(val):
        return np.nan
    val = str(val).lower().strip()
    return re.sub(r'\W+', '', val)

def get_sampled_values(df, col_name, sample_size=50):
    values = df[col_name].dropna().astype(str).map(canonicalize).unique()
    values = sorted(values)
    if len(values) <= sample_size:
        return set(values)
    third = sample_size // 3
    return set(values[:third] + values[len(values)//2 - third//2 : len(values)//2 + third//2] + values[-third:])

def load_all_tables(folder_path):
    tables = {}
    for file in Path(folder_path).glob("*.csv"):
        name = re.sub(r"\.0$", "", file.stem.strip().lower())
        try:
            df = pd.read_csv(file, dtype=str, low_memory=False)
            tables[name] = df.applymap(lambda x: str(x).strip().lower() if pd.notnull(x) else x)
        except Exception as e:
            print(f"Error loading {file.name}: {e}")
    return tables

def verify_match_with_samples(row, tables):
    pk_table = row["pk_table"].strip().lower()
    fk_table = row["fk_table"].strip().lower()
    pk_col = row["pk_column"]
    fk_col = row["fk_column"]

    try:
        pk_df = tables[pk_table]
        fk_df = tables[fk_table]
        if pk_col not in pk_df.columns or fk_col not in fk_df.columns:
            return 0.0

        pk_vals = get_sampled_values(pk_df, pk_col)
        fk_vals = get_sampled_values(fk_df, fk_col)

        if not fk_vals:
            return 0.0

        match = len(fk_vals & pk_vals) / len(fk_vals)
        return round(match, 3)
    except Exception as e:
        print(f"Match error {pk_table}.{pk_col} -> {fk_table}.{fk_col}: {e}")
        return 0.0

def verify_fk_matches_with_overlap(matches_df, data_folder, min_score=5.5, sample_size=50):
    tables = load_all_tables(data_folder)

    filtered_matches = matches_df[matches_df["score"] >= min_score].copy()

    filtered_matches["verified_overlap"] = filtered_matches.apply(
        lambda row: verify_match_with_samples(row, tables), axis=1
    )

    filtered_matches["verified_label"] = filtered_matches["verified_overlap"].apply(
        lambda x: "verified_strong" if x > 0.8 else ("weak_match" if x > 0.4 else "unverified")
    )

    # Optional: merge back if needed
    enriched_df = matches_df.merge(
        filtered_matches[["pk_table", "pk_column", "fk_table", "fk_column", "verified_overlap", "verified_label"]],
        on=["pk_table", "pk_column", "fk_table", "fk_column"],
        how="left"
    )

    return enriched_df

In [24]:
enriched_matches_df = verify_fk_matches_with_overlap(matches_df, DATA_FOLDER, min_score=5.5)

In [26]:
final_df = enriched_matches_df.merge(
    matches_df[["pk_table", "pk_column", "fk_table", "fk_column"]],
    on=["pk_table", "pk_column", "fk_table", "fk_column"],
    how="left"
)

In [28]:
final_df.to_csv('inter_table_relations_scoring.csv',index= False)